In [1]:
import re
import math
import nltk
import pandas as pd
import mysql.connector
from datetime import date
from html import unescape
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
def preprocessing(data):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    stopword = StopWordRemoverFactory().create_stop_word_remover()

    def cleansing(row):
        text = re.sub(r'&[^\s;&]+;', '', unescape(row['text']))
        html_pattern = re.compile('<.*?>')
        text = html_pattern.sub(r' ', text)
        text = re.sub(
            r'((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)
        return text
    
    def caseFolding(row):
        text = row['text'].lower()
        return text

    def tokenizing(row):
        tokenized = word_tokenize(str(row['text']))
        return tokenized

    def stemming(row):
        stemmed = [stemmer.stem(token) for token in row['text']]
        stemmed = " ".join(stemmed)
        return stemmed

    def stopwording(row):
        stopworded = stopword.remove(row['text'])
        return stopworded

    data['text'] = data.apply(cleansing, axis=1)
    data['text'] = data.apply(caseFolding, axis=1)
    data['text'] = data.apply(tokenizing, axis=1)
    data['text'] = data.apply(stemming, axis=1)
    data['text'] = data.apply(stopwording, axis=1)

    return data

In [6]:
vacancies = [
    "Lowongan Kerja Web Developer di Perusahaan ABC",
    "Dibutuhkan Software Engineer untuk Proyek Inovatif",
    "Lowongan Kerja Data Analyst dengan Gaji Menarik",
    "Dicari UI/UX Designer berpengalaman untuk Startup",
    "Lowongan Kerja Mobile App Developer Full-time",
    "Dibutuhkan Software Engineer dengan pengalaman di bidang pengembangan web.",
    "Dicari UI/UX Designer yang kreatif dan berpengalaman.",
    "Perusahaan mencari Data Scientist untuk menganalisis data dan membuat model prediktif.",
    "Dibutuhkan Digital Marketing Specialist untuk mengelola kampanye pemasaran online.",
    "Perusahaan mencari Product Manager yang memiliki pengalaman di industri teknologi.",
    "Dicari Content Writer yang kreatif dan mampu menulis konten berkualitas.",
    "Perusahaan membutuhkan Frontend Developer untuk mengembangkan antarmuka pengguna.",
    "Dibutuhkan HR Manager dengan pengalaman dalam manajemen sumber daya manusia.",
    "Dicari Graphic Designer yang mampu menciptakan desain visual yang menarik.",
    "Perusahaan mencari Business Analyst untuk menganalisis kebutuhan bisnis.",
    "Dibutuhkan Sales Executive untuk menjalin hubungan bisnis dengan klien.",
    "Dicari Full Stack Developer yang memiliki pengetahuan luas tentang teknologi terkini.",
    "Perusahaan membutuhkan Customer Service Representative yang ramah dan efisien.",
    "Dibutuhkan Project Manager untuk mengelola proyek dan tim dengan baik.",
    "Dicari Social Media Specialist untuk mengelola kehadiran online perusahaan.",
    "Perusahaan mencari Android Developer yang berpengalaman dalam pengembangan aplikasi.",
    "Dibutuhkan Accountant untuk mengelola keuangan perusahaan.",
    "Dicari Network Administrator yang memiliki pemahaman tentang jaringan komputer.",
    "Perusahaan membutuhkan Legal Counsel untuk memberikan nasihat hukum.",
    "Dibutuhkan UI/UX Researcher untuk mengumpulkan dan menganalisis data pengguna."
]
itemPerPage = 5
page = 1
minWeight = 0.15
keyword = "DIBUTUHKAN Untuk"


dframeVacancies = pd.DataFrame(vacancies, columns=["text"])
dframeVacancies = dframeVacancies.dropna()
dframeVacancies = preprocessing(dframeVacancies)

dframeKeyword = pd.DataFrame([keyword], columns=["text"])
dframeKeyword = preprocessing(dframeKeyword)

In [7]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(dframeVacancies['text'])
new_tfidf_vector = vectorizer.transform(dframeKeyword["text"])

vectorizerWithoutNorm = TfidfVectorizer(norm=None)
tfidf_matrixWithoutNorm = vectorizerWithoutNorm.fit_transform(dframeVacancies['text'])
new_tfidf_vectorWithoutNorm = vectorizerWithoutNorm.transform(dframeKeyword["text"])

countVectorizer = CountVectorizer()
tf = countVectorizer.fit_transform(dframeVacancies['text'])
tf_keyword = countVectorizer.transform(dframeKeyword["text"])

document_frequencies = tfidf_matrix.astype(bool).sum(axis=0).A1
words = vectorizer.get_feature_names_out()
idf_values = vectorizer.idf_

document_frequenciesWithoutNorm = tfidf_matrixWithoutNorm.astype(bool).sum(axis=0).A1
wordsWithoutNorm = vectorizerWithoutNorm.get_feature_names_out()
idf_valuesWithoutNorm = vectorizerWithoutNorm.idf_